# Vertex AI Gemini API の基本的な利用方法と Gemini が提供する Grounding 手法を理解する

## はじめに
Grounding は一般的に RAG とも言われる特定の情報を参照して回答することでハルシネーションを軽減したり、企業独自の内容で質問に回答させる手法です。Gemini ではネイティブに Grounding を前提とした API を提供しており、簡単なコーディングでRAGを実装することが可能です。

## 環境セットアップ
Python の前提パッケージを導入します。Vertex AI Gemini API を利用する場合は 1 つ目のライブラリだけが必須となります。LangChain を利用する場合は 2 つ目以降のライブラリも導入します。

In [ ]:
%pip install --upgrade google-genai langchain-google-vertexai langchain-google-community google-cloud-discoveryengine

次のスクリプトでカーネルを再起動します。

 - コンシューマー向け Colab の場合、上記のログに"RESTART RUNTIME"ボタンをが表示された場合、ボタンを押してカーネルをリスタートできます。
 - Vertex AI Workbench の場合、メニューよりカーネルのリスタートを実行できます。

In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

また、このコードを実行するユーザーに`Vertex AI ユーザー`の IAM ロールを付与します。

Colab の場合、以下を実行し Vertex AI API のユーザー権限をもつアカウントでログインします。 Vertex AI Workbench の場合はスキップされます。

In [ ]:
import sys

if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

環境変数などを定義します。 Google Cloud のプロジェクト ID を指定してください。

In [ ]:
PROJECT_ID = "<your_project_id>"  # @param {type: "string"}
LOCATION = "global"  # @param {type: "string"}
MODEL_ID = "gemini-2.0-flash-001"  # @param {type: "string"}

Gemini API を初期化します。

In [ ]:
from IPython.display import Markdown, HTML
from google import genai
from google.genai.types import Part, GenerateContentConfig, Tool, GoogleSearch, VertexAISearch, Retrieval

client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

Gemini の質問するプロンプトを定義します。

In [ ]:
PROMPT = "Google Cloud の2023年と2024年のRevenueは?"
#PROMPT = "Cloud Spannerの特徴は？"

## Gemini の知識だけで回答

最初に Gemini の知識だけでの回答を確認します。基盤モデル学習時に存在しない最新の情報にはハルシネーションが含まれる場合があります。

In [ ]:
#generate_content()利用
#response = client.models.generate_content(model=MODEL_ID, contents=PROMPT)
#Chat API利用
chat = client.chats.create(model=MODEL_ID)
response = chat.send_message(PROMPT)
Markdown(response.text)

Google Cloudの収益に関する情報は、Googleの親会社であるAlphabetの決算報告書で確認できます。

*   **2023年:** Google Cloudの収益は**330億7600万ドル**でした。
*   **2024年第1四半期:** Google Cloudの収益は**95億700万ドル**でした。

2024年全体の収益は、まだ発表されていません。Alphabetの四半期ごとの決算報告書で更新される予定です。

## Gemini にPDFファイルを直接渡して回答

PDF ファイルを取得します。

In [ ]:
!curl -o ref.pdf https://abc.xyz/assets/77/51/9841ad5c4fbe85b4440c47a4df8d/goog-10-k-2024.pdf
#!curl -o ref.pdf https://lp.cloudplatformonline.com/rs/808-GJW-314/images/Database_OnAir_q2_0409_Session.pdf
with open("ref.pdf", "rb") as f:
    pdfFile = f.read()

PDF ファイルとプロンプトを直接 Gemini に入力します。

ここでのポイントは、Gemini は PDF の認識に対応しているため、PDF ファイルを別途パースしてテキスト部分を取り出すする必要はありません。また、マルチモーダルで図表を認識するため、図表固有の抽出処理も必要ありません。

In [ ]:
#generate_content()利用
#response = client.models.generate_content(model=MODEL_ID, contents=[
#Chat API利用
chat = client.chats.create(model=MODEL_ID)
response = chat.send_message([
    Part.from_bytes(data=pdfFile, mime_type="application/pdf"),
    Part.from_text(text=PROMPT),
])
Markdown(response.text)

Google Cloudの2023年のRevenueは33,088百万ドルで、2024年は43,229百万ドルです。

## Google Search による Grounding

次は、Google Search で Grounding した回答を取得します。

Gemini による回答だけでなく、検索によって参照した関連情報も確認することが可能です。

In [ ]:
google_search_tool = Tool(google_search=GoogleSearch())

#response = client.models.generate_content(model=MODEL_ID, contents=PROMPT, config=GenerateContentConfig(tools=[google_search_tool]))
chat = client.chats.create(model=MODEL_ID, config=GenerateContentConfig(tools=[google_search_tool]))
response = chat.send_message(PROMPT)

Markdown(response.text)

Google Cloud の収益について、2023年と2024年の情報が見つかりました。

**2023年**

*   通年の収益は331億ドルでした。
*   第4四半期の収益は、前年同期比で26%増加しました。

**2024年**

*   通年の収益は480億ドルでした。
*   第4四半期の収益は120億ドルで、前年同期比で30%増加しました。
*   Google CloudとYouTubeを合わせた年間収益は1,100億ドルの規模に達しました。

2024年は、Google Cloudにとって非常に成長の著しい年であり、特にAI関連のサービスが大きく貢献しました。


In [ ]:
for groundingChunk in response.candidates[0].grounding_metadata.grounding_chunks:
    print(groundingChunk)

HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content)

retrieved_context=None web=GroundingChunkWeb(title='crn.com', uri='https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqALWrrEbp_lkhGkgiVNid11cI3BDyZZ90FogiLhspJmKco1KiA9bhUoeEOtWWMZwuCsjNnfqr4NtPdIE3PXX2PfZ9aOsmWPuqshrQo8hUIspU2zOSuuLfm39AC1LQC0GYCquE2ns8CYIZ7iXatS1v_8Fl9mrBDMxzSSXxDWhgiLgxEONw6XUxnTpAU15KYImQcnry6vDBBSOAm4=')
retrieved_context=None web=GroundingChunkWeb(title='mobileworldlive.com', uri='https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqAJmBnBWgUE14J0GlAcndbRafAKcBtjeSAe2qGGKF5UG67d4PCJTBCRaahE6gzL90-oOvVgevu6mRdzNv4ZUqXi6vxSngLwV7qVM5u7D-_mJH3D_AqI_LgYTUjcxWB4dcAfgIw0UlJpRx8XVmq92stelGU4HNeAv6MrlG0Qaz6_SyNG4mwtPEg==')
retrieved_context=None web=GroundingChunkWeb(title='crn.com', uri='https://vertexaisearch.cloud.google.com/grounding-api-redirect/AWQVqAI_LPWSVC8vTG_NhIPQebsYo5o7YzV6YSpFIGCBies4LrCl_qEPjt71Q0wffWA9_n5icZ7PMppAUw4YQepERNG1FuWy7UbFmYo8OSofPSLBzI-X5lytU_DXcOQIhmseC9k_bdWLeONQKEQZimt6rPUbNIYFUX-JDm72JgXDawN3WGhKlUSd0rm65q1w

## Vertex AI Search による Grounding

次は、Vertex AI Search で Grounding した回答を取得します。

最初に Cloud Storage にバケットを作成し、回答させたいコンテンツを含む PDF ファイルをアップロードしておきます。

Vertex AI Search の「[検索データストアを作成する - Cloud Storage からインポートする](https://cloud.google.com/generative-ai-app-builder/docs/create-data-store-es?hl=ja#cloud-storage)」の手順に従って Vertex AI Search のデータストアをセットアップします。インデックスの作成にしばらく時間がかかります。Vertex AI Search の検索アプリを同時に作成した場合は、プレビュー機能で検索が行えるか確認することが可能です。

最後に、Cloud Console の Agent Builder - データストア より、作成したデータストアの ID を確認し、以下の変数に定義します。

In [ ]:
DATA_STORE_LOCATION = "global"
DATA_STORE_ID = "<datastore_id>"  # @param {type: "string"}

Gemini 自身による回答だけでなく、Vertex AI Search から取得した内容で回答できていることを確認します。

In [ ]:
vertex_ai_search_tool = Tool(
    retrieval=Retrieval(
        vertex_ai_search=VertexAISearch(
            datastore=f"projects/{PROJECT_ID}/locations/{DATA_STORE_LOCATION}/collections/default_collection/dataStores/{DATA_STORE_ID}"
        )
    )
)

#response = client.models.generate_content(model=MODEL_ID, contents=PROMPT, config=GenerateContentConfig(tools=[vertex_ai_search_tool]))
chat = client.chats.create(model=MODEL_ID, config=GenerateContentConfig(tools=[vertex_ai_search_tool]))
response = chat.send_message(PROMPT)

Markdown(response.text)

Google Cloudの収益は以下の通りです。

*   **2023年:** 330億8800万ドル
*   **2024年:** 432億2900万ドル

また、回答作成時に参照した関連情報も確認することが可能です。

In [ ]:
for groundingChunk in response.candidates[0].grounding_metadata.grounding_chunks:
    print(groundingChunk)

retrieved_context=GroundingChunkRetrievedContext(text='Google subscriptions, platforms, and devices\nGoogle subscriptions, platforms, and devices revenues increased $5.7 billion from 2023 to 2024. The growth was\nprimarily driven by an increase in subscription revenues, largely from growth in the number of paid subscribers for\nYouTube services followed by Google One. Google Cloud\nGoogle Cloud revenues increased $10.1 billion from 2023 to 2024 primarily driven by growth in Google Cloud\nPlatform largely from infrastructure services. Revenues by Geography\nThe following table presents revenues by geography as a percentage of revenues, determined based on the\naddresses of our customers: Year Ended December 31,\n2023 2024 United States 47 % 49 % EMEA 30 % 29 % APAC 17 % 16 % Other Americas 6 % 6 % Hedging gains (losses) 0 % 0 % For additional information, see Note 2 of the Notes to Consolidated Financial Statements included in Item 8 of this\nAnnual Report on Form 10-K.\nUse of Non-GAAP

## (option) LangChain を利用した RAG

ここでは LangChain を利用して、Vertex AI Search を利用した Gemini による RAG サンプルを示します。

最初に、Gemini の知識だけでの回答を確認します。

In [ ]:
import vertexai
from langchain_google_vertexai import ChatVertexAI

vertexai.init(project=PROJECT_ID, location="us-central1")

llm = ChatVertexAI(model_name=MODEL_ID)
response = llm.invoke(PROMPT)
Markdown(response.content)

Google Cloud の Revenue は以下の通りです。

*   **2023年:** 330億ドル（約4兆8000億円）

*   **2024年第1四半期:** 95億7000万ドル（約1兆4800億円）

Vertex AI Search のラッパーである VertexAISearchRetriever の動きを確認します。


In [ ]:
from langchain_google_community import VertexAISearchRetriever

retriever = VertexAISearchRetriever(
    project_id=PROJECT_ID, location_id=DATA_STORE_LOCATION, data_store_id=DATA_STORE_ID, get_extractive_answers=True, max_extractive_answer_count=3, max_documents=5
)

result = retriever.invoke(PROMPT)
for doc in result:
    print(doc)

/usr/local/lib/python3.11/dist-packages/langchain_google_community/vertex_ai_search.py:364: UserWarning: Beta features are configured but beta=False. The following beta features will be ignored:['custom_embedding_ratio']
  warnings.warn(


page_content='AWS サービスや Azure サービスと <b>Google Cloud</b> を比較する最終更新日: <b>2023 年</b> 8 月 5 日次の表に、一般提供 (https://<b>cloud</b>.<b>google</b>.com/products?hl=ja#general-availability)の <b>Google Cloud</b> サービスと、それに対応するアマゾン ウェブ サービス（AWS）または Microsoft Azure のサービスを示します。この表は、サービスタイプ、機能、サービス名などのキーワードでフィルタできます。今後も最新の情報をお届けするために、をお寄せください。 サービス カテゴリサービスタイプ <b>Google Cloud</b> プロダクト<b>Google Cloud</b> サービスの概要AWS サービスAzure サービスAI &amp; ML <b>クラウド</b>費用の最適化Recommender (https://<b>cloud</b>.<b>google</b>.com/recommender?hl=ja) プロアクティブで容易に実行可能な推奨事項を提示することで、 <b>Google Cloud</b> の使用量を最適化します。 AWS 費用の最適化Azure Cost Management AI &amp; ML 会話型インターフェースDialogflow (https://<b>cloud</b>.<b>google</b>.com/dialogflow?hl=ja) 最先端の仮想エージェントを備えた、自然な会話を実現する AI。 Amazon Lex Azure Conversational AI AI &amp; ML ドキュメントの理解Document AI (https://<b>cloud</b>.<b>google</b>.com/document-ai?hl=ja) 大規模なデータ キャプチャを自動化してドキュメントの処理費用を削減します。 Amazon Textract Azure Form Recognizer AI &amp; ML 画像認識Vision AI (https://<b>cloud</b>.<b>google</b>.com

VertexAISearchRetriever と Gemini を組み合わせた RAG による回答を確認します。

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_google_vertexai import ChatVertexAI

prompt = ChatPromptTemplate.from_template(
    """次のコンテキスト情報を利用して、最後の質問に答えてください。回答は300字程度で回答してください。

Context: {context}

Question: {question}"""
)

llm = ChatVertexAI(model_name=MODEL_ID)
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
response = chain.invoke(PROMPT)
Markdown(response)

Google Cloud の Revenue は、2023年には33088百万ドル、2024年には43229百万ドルでした。2023年から2024年にかけて、Google Cloud の Revenue は101億ドル増加しました。この成長は主に Google Cloud Platform、特にインフラストラクチャサービスによるものです。


## Code Execution でデータ分析

分析用の CSV ファイルを取得します。

In [ ]:
!curl -o tokyo-chika-2025.csv https://storage.googleapis.com/public4llm/tokyo-chika-2025.csv
with open("tokyo-chika-2025.csv", "rb") as f:
    csvFile = f.read()

ファイルのレイアウトを確認します。

In [24]:
import pandas as pd

df = pd.read_csv('tokyo-chika-2025.csv')
df.head()

,所在・地番,住居表示,利用現況,駅名,駅距離,都市計画用途地域,対前年変動率,価格R06,価格R07
0,東京都 千代田区三番町６番２５,_,共同住宅,半蔵門,500,2住居,10.0,3600000,3960000
1,東京都 千代田区紀尾井町３番２７外,紀尾井町３－３２,共同住宅,永田町,300,2住居,11.9,2260000,2530000
2,東京都 千代田区六番町６番１外,_,共同住宅,四ツ谷,330,2住居,10.0,4390000,4830000
3,東京都 千代田区富士見１丁目８番６,富士見１－４－１２,共同住宅,飯田橋,360,1住居,13.2,1740000,1970000
4,東京都 千代田区九段北２丁目６番２６,九段北２－３－２５,共同住宅,九段下,450,1住居,10.2,3340000,3680000


Gemini の Code Execution 機能を利用してデータ分析を行います。

In [ ]:
from google.genai.types import ToolCodeExecution

code_execution_tool = Tool(code_execution=ToolCodeExecution())
response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        Part.from_bytes(data=csvFile, mime_type="text/csv"),
        Part.from_text(text="地価が高い土地上位20を表示して"),
    ],
    config=GenerateContentConfig(
        tools=[code_execution_tool],
        temperature=0,
    ),
)

response

In [27]:
Markdown(response.executable_code)

import pandas as pd

# Load the CSV file into a pandas DataFrame
df = pd.read_csv("input_file_0.csv")

# Sort the DataFrame by the '価格R07' column in descending order
df_sorted = df.sort_values(by='価格R07', ascending=False)

# Select the top 20 rows
df_top20 = df_sorted.head(20)

# Print the selected data
print(df_top20)


In [28]:
Markdown(response.code_execution_result)

                   所在・地番        住居表示         利用現況     駅名  駅距離 都市計画用途地域  対前年変動率     価格R06     価格R07
88       東京都　中央区銀座４丁目２番４     銀座４－５－６           店舗     銀座    0       商業     8.6  55700000  60500000
107   東京都　中央区銀座５丁目１０３番１６     銀座５－４－３       店舗兼事務所     銀座    0       商業     8.6  47600000  51700000
95     東京都　中央区銀座２丁目２番１９外     銀座２－６－７       店舗兼事務所  銀座一丁目    0       商業     8.6  40900000  44400000
89      東京都　中央区銀座７丁目１番２外    銀座７－９－１９           店舗     銀座  260       商業     8.0  39900000  43100000
264   東京都　新宿区新宿３丁目８０７番１外    新宿３－２４－１       店舗兼事務所     新宿    0       商業     5.3  38000000  40000000
253   東京都　新宿区新宿３丁目３０番１３外   新宿３－３０－１１       店舗兼事務所  新宿三丁目    0       商業     4.6  36900000  38600000
8     東京都　千代田区丸の内２丁目２番１外    丸の内２－４－１       事務所兼店舗     東京    0       商業     0.8  36800000  37100000
845    東京都　渋谷区宇田川町７７番１４外    宇田川町２３－３    銀行、店舗兼事務所     渋谷  150       商業     7.0  31300000  33500000
84    東京都　中央区銀座４丁目１０３番１外    銀座４－２－１５       店舗兼事務所     銀座    0       商業     8.4  29600000  32100000
68     東京都　中央区銀座６丁目４番１３外     銀座６－８－３           店舗     銀座  150       商業     5.1  29300000  30800000
27    東京都　千代田区大手町２丁目４番２外    大手町２－２－１   店舗、事務所兼駐車場    大手町    0       商業     1.0  29100000  29400000
849  東京都　渋谷区神宮前４丁目２６番４３外  神宮前４－２６－１８   店舗、事務所兼住宅等  明治神宮前  150       商業     1.1  28200000  28500000
25     東京都　千代田区丸の内３丁目２番外    丸の内３－３－１  事務所、店舗兼駐車場等     東京  480       商業     0.7  26900000  27100000
837   東京都　渋谷区道玄坂２丁目３６番１０   道玄坂２－６－１７   店舗、事務所兼映画館     渋谷  200       商業     5.8  24200000  25600000
71      東京都　中央区京橋１丁目１番１外           _          建築中     東京  300       商業     3.5  22900000  23700000
109  東京都　中央区八重洲１丁目１０５番５３   八重洲１－５－２０   店舗、事務所兼駐車場     東京  280       商業     2.6  23000000  23600000
234     東京都　新宿区西新宿１丁目７番３    西新宿１－７－１       店舗兼事務所     新宿    0       商業     4.7  21200000  22200000
101    東京都　中央区日本橋２丁目１番１外   日本橋２－１－１０       店舗兼事務所    日本橋    0       商業     3.3  21300000  22000000
856    東京都　渋谷区神宮前１丁目１３番５  神宮前１－１３－１１    店舗、事務所兼住宅     原宿  190       商業     4.3  20900000  21800000
31    東京都　千代田区有楽町１丁目１２番２    有楽町１－１－３   劇場、事務所兼店舗等    日比谷  100       商業     0.9  21500000  21700000
